In [10]:
# 라이브러리 추가
import urllib.request
import json
import pandas as pd

In [26]:
# 네이버 데이터랩 API를 이용한 검색량 조회 함수 (최대 5개)
def datalab_api(keywords):

    # 데이터랩 API
    client_id = "89L30Y3xCshEctCoab1v"
    client_secret = "aIoL3WXAsj"

    url = "https://openapi.naver.com/v1/datalab/search"

    body = {
        "startDate":"2018-12-31",
        "endDate":"2019-11-30",
        "timeUnit":"month", # date: 일간, week: 주간, month: 월간
        "keywordGroups":[
            {"groupName":str(keywords[0]),"keywords":[str(keywords[0])]},
            {"groupName":str(keywords[1]),"keywords":[str(keywords[1])]},
            {"groupName":str(keywords[2]),"keywords":[str(keywords[2])]},
            {"groupName":str(keywords[3]),"keywords":[str(keywords[3])]},
            {"groupName":str(keywords[4]),"keywords":[str(keywords[4])]},
        ],
    }
    body = json.dumps(body)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    request.add_header("Content-Type","application/json")
    response = urllib.request.urlopen(request, data=body.encode("utf-8"))
    json_obj = json.loads(response.read().decode('utf-8'))

    # 데이터프레임
    df1 = pd.DataFrame(json_obj['results'][0]['data']); df1.columns=['기간', str(keywords[0])]
    df2 = pd.DataFrame(json_obj['results'][1]['data']); df2.columns=['기간', str(keywords[1])]
    df3 = pd.DataFrame(json_obj['results'][2]['data']); df3.columns=['기간', str(keywords[2])]
    df4 = pd.DataFrame(json_obj['results'][3]['data']); df4.columns=['기간', str(keywords[3])]
    df5 = pd.DataFrame(json_obj['results'][4]['data']); df5.columns=['기간', str(keywords[4])]
    
    # 데이터프레임 병합
    df2 = df1.merge(df2, how='outer', on='기간').fillna(0)
    df3 = df2.merge(df3, how='outer', on='기간').fillna(0)
    df4 = df3.merge(df4, how='outer', on='기간').fillna(0)
    df5 = df4.merge(df5, how='outer', on='기간').fillna(0)
    
    return df5

In [27]:
kw_list = ['국립공원관리공단', '국립공원공단', '국립공원', '공단', '심평원']

In [32]:
df = datalab_api(kw_list)
df

,기간,국립공원관리공단,국립공원공단,국립공원,공단,심평원
0,2018-12-01,45.01624,0.09332,12.45697,14.01439,60.20385
1,2019-01-01,42.97401,2.43249,12.40243,14.66888,71.18703
2,2019-02-01,36.67644,4.16686,12.51272,12.26668,57.21384
3,2019-03-01,32.75439,6.89872,14.12105,14.40708,87.16366
4,2019-04-01,33.03922,9.07548,17.91947,14.12469,100.00000
5,2019-05-01,35.65230,8.39554,20.11198,13.52474,74.13099
6,2019-06-01,42.87463,11.49343,19.24055,13.63140,69.96412
7,2019-07-01,48.18684,17.37892,22.99413,13.47869,66.79715
8,2019-08-01,33.42827,13.32476,20.17258,12.69816,58.77733
9,2019-09-01,31.95205,10.35414,18.96422,12.70058,60.79774


In [73]:
rv = [10380, 10310, 10410, 11740, 14890, 16710, 16310, 19430, 16980, 16020, 19260, 14140]
ev = df.loc[:, ['국립공원']].values.flatten().tolist()

In [81]:
ev

[12.45697,
 12.40243,
 12.51272,
 14.12105,
 17.91947,
 20.11198,
 19.24055,
 22.99413,
 20.17258,
 18.96422,
 22.80869,
 16.73413]

In [67]:
# scipy.stats.linregress
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(ev,rv)

pv = ev * slope + intercept
pv

TypeError: can't multiply sequence by non-int of type 'numpy.float64'